# Planetary Boundary LCIA Example

This notebook demonstrates how to:
1. Install pb-aesa LCIA methods to Brightway
2. Run an LCA on a process
3. Calculate and print LCIA results using the Planetary Boundary methods

## 1. Import Required Packages

In [ ]:
import bw2data as bd
import bw2calc as bc
import pb_aesa
import pandas as pd

## 2. Initialize Brightway Project

Set up your Brightway project. Adjust the project name and database names according to your setup.

In [ ]:
# Set the current Brightway project
bd.projects.set_current('PB_AESA')

# Load databases
bio = bd.Database("ecoinvent-3.10.1-biosphere")
ei = bd.Database("ecoinvent-3.10.1-cutoff")

print(f"Project: {bd.projects.current}")
print(f"Databases loaded: {list(bd.databases)}")

## 3. Install Planetary Boundary LCIA Methods

This step creates and registers all planetary boundary LCIA methods in Brightway.

In [ ]:
# Create all LCIA methods (standard + nitrogen cycle)
pb_aesa.implement_lcia_methods(bio)

# Verify methods were created
pb_methods = [met for met in bd.methods if "Planetary Boundaries" in str(met)]
print(f"\nInstalled {len(pb_methods)} Planetary Boundary LCIA methods:")
for method in pb_methods:
    print(f"  - {method}")

## 4. Select a Process for Analysis

Choose a process from the ecoinvent database to analyze.

In [ ]:
# Example: Select wheat grain production in Germany
# You can modify this to select any process you're interested in

process = [act for act in ei 
           if 'DE' in act['location'] 
           and 'wheat grain production' in act['name']][0]

print(f"Selected process:")
print(f"  Name: {process['name']}")
print(f"  Location: {process['location']}")
print(f"  Unit: {process['unit']}")
print(f"  Code: {process['code']}")

## 5. Run LCA with Single Method

First, let's run a simple LCA with a single planetary boundary method.

In [ ]:
# Select Climate Change method
climate_method = ('Planetary Boundaries', 'Climate Change')

# Create and run LCA
lca = bc.LCA({process: 1}, climate_method)
lca.lci()
lca.lcia()

# Get result
climate_score = lca.score

print(f"\nLCA Result for {climate_method[1]}:")
print(f"  Score: {climate_score:.6f}")
print(f"  Unit: {bd.methods[climate_method].get('unit', 'N/A')}")

## 6. Run Multi-LCA with All Planetary Boundary Methods

Now let's run an LCA with all planetary boundary methods simultaneously.

In [ ]:
# Get all planetary boundary methods
pb_methods = [met for met in bd.methods if "Planetary Boundaries" in str(met)]

# Define functional unit
functional_unit = {
    "process_1": {ei.get(process["code"]).id: 1.0},
}

# Configure MultiLCA
config = {
    "impact_categories": pb_methods
}

# Get data objects
data_objs = bd.get_multilca_data_objs(
    functional_units=functional_unit, 
    method_config=config
)

# Run MultiLCA
mlca = bc.MultiLCA(
    demands=functional_unit, 
    method_config=config, 
    data_objs=data_objs
)
mlca.lci()
mlca.lcia()

print("\nMulti-LCA completed successfully!")
print(f"Analyzed {len(mlca.scores)} impact categories")

## 7. Display LCIA Results

Show the raw LCIA scores for each planetary boundary category.

In [ ]:
# Create a DataFrame with the results
results_data = []
for method_key, score in mlca.scores.items():
    category = method_key[0][1]
    unit = bd.methods[method_key[0]].get('unit', 'N/A')
    results_data.append({
        'Category': category,
        'Score': score,
        'Unit': unit
    })

results_df = pd.DataFrame(results_data)
results_df = results_df.sort_values('Score', ascending=False)

print("\n" + "="*80)
print("LCIA RESULTS - Planetary Boundaries")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

## 8. Calculate Exploitation of Safe Operating Space

Normalize the results by the Safe Operating Space thresholds to see how much of the planetary boundary is being exploited.

In [ ]:
# Calculate exploitation of Safe Operating Space
exploit = pb_aesa.calculate_exploitation_of_SOS(mlca.scores)

# Create DataFrame for better display
exploit_data = []
for method_key, value in exploit.items():
    category = method_key[0][1]
    threshold = pb_aesa.SAFE_OPERATING_SPACE.get(category, None)
    exploit_data.append({
        'Category': category,
        'SOS Threshold': threshold,
        'Impact Score': mlca.scores[method_key],
        'SOS Exploitation': value if value is not None else 'N/A'
    })

exploit_df = pd.DataFrame(exploit_data)
exploit_df = exploit_df.sort_values('SOS Exploitation', ascending=False, 
                                     key=lambda x: pd.to_numeric(x, errors='coerce'))

print("\n" + "="*90)
print("EXPLOITATION OF SAFE OPERATING SPACE")
print("="*90)
print(exploit_df.to_string(index=False))
print("="*90)
print("\nNote: SOS Exploitation > 1.0 means the safe operating space is exceeded")

## 9. Visualize Results

Create a bar chart showing the exploitation of Safe Operating Space for each planetary boundary.

In [ ]:
# Plot exploitation of Safe Operating Space
pb_aesa.plot_exploitation_of_SOS(exploit)

## 10. Summary and Interpretation

Let's create a summary showing which boundaries are most stressed.

In [ ]:
# Identify categories with highest exploitation
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

# Filter out N/A values and convert to numeric
valid_exploit = {k: v for k, v in exploit.items() if v is not None}

if valid_exploit:
    # Sort by exploitation value
    sorted_exploit = sorted(valid_exploit.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\nProcess analyzed: {process['name']} ({process['location']})")
    print(f"Functional unit: 1 {process['unit']}\n")
    
    print("Top 3 most stressed planetary boundaries:")
    for i, (method_key, value) in enumerate(sorted_exploit[:3], 1):
        category = method_key[0][1]
        status = "⚠️ EXCEEDED" if value > 1.0 else "✓ Within bounds"
        print(f"{i}. {category}: {value:.6f} ({status})")
    
    # Count exceeded boundaries
    exceeded = sum(1 for v in valid_exploit.values() if v > 1.0)
    print(f"\nTotal boundaries exceeded: {exceeded} out of {len(valid_exploit)}")
else:
    print("No valid exploitation values calculated.")

print("="*80)

## Additional Analysis Options

### Access Safe Operating Space Thresholds

In [ ]:
# Display all Safe Operating Space thresholds
print("\nSafe Operating Space Thresholds:")
for category, threshold in pb_aesa.SAFE_OPERATING_SPACE.items():
    unit = pb_aesa.PLANETARY_BOUNDARY_UNITS.get(category, 'N/A')
    print(f"  {category}: {threshold} ({unit})")

### Compare Multiple Processes

You can extend this notebook to compare multiple processes by running the analysis for different activities and comparing their exploitation values.

In [ ]:
# Example: Compare different agricultural processes
# This is a template - adjust according to your needs

# processes_to_compare = [
#     'wheat grain production',
#     'maize grain production',
#     'potato production'
# ]

# comparison_results = {}
# for process_name in processes_to_compare:
#     proc = [act for act in ei if process_name in act['name'] and 'DE' in act['location']][0]
#     # Run LCA and calculate exploitation...
#     # Store results in comparison_results

print("\nTo compare multiple processes, uncomment and modify the code above.")